In [ ]:
import pandas as pd 
import numpy as np 

from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
#load our datasets 
train = pd.read_csv('/content/gdrive/My Drive/analyticsvidhya/hranalytics/train.csv')
test = pd.read_csv('/content/gdrive/My Drive/analyticsvidhya/hranalytics/test.csv')
submission = pd.read_csv('/content/gdrive/My Drive/analyticsvidhya/hranalytics/submission.csv')

In [ ]:
train.head()

In [ ]:
ntrain = train.shape[0]
ntest = test.shape[0]
all_data = pd.concat((train,test)).reset_index(drop=True)
print(f'The shape of the combined dataframe is: {all_data.shape}')

In [ ]:
all_data.info()

In [ ]:
all_train.describe().T


In [ ]:
all_train.describe(include=['O']).T

In [ ]:
all_data.isnull().sum()

In [ ]:
import seaborn as sns 
sns.countplot(all_data.is_promoted)

In [ ]:
# class is heavily imbalanced 
all_data['education'] = all_data['education'].fillna(all_data['education'].mode()[0])
all_data['previous_year_rating'] = all_data['previous_year_rating'].fillna(all_data['previous_year_rating'].mode()[0])
all_data['is_promoted'] = all_data['is_promoted'].fillna(0)

In [ ]:
pip install category_encoders

In [ ]:
pip install catboost

In [ ]:
all_data.drop(['employee_id'], 1, inplace = True)

In [ ]:
region = all_data.groupby('region').mean()
region.drop('is_promoted', 1, inplace = True)
cols = []
for i in region.columns:
    if i != 'region':
        region[i+'_reg_mean_all'] = region[i]
        region.drop(i, 1, inplace = True)
        cols.append(i+'_reg_mean_all')

data = pd.merge(all_data, region, on='region', how = 'left')
for col in cols: data[col+'_freq'] = data[col].map(data[col].value_counts().to_dict())/len(data)

In [ ]:
data.head()

In [ ]:
import category_encoders as ce
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, data['is_promoted'], test_size=0.3,
                                                    random_state=0)
X_train.shape, X_test.shape

ord_enc = ce.OrdinalEncoder(cols=['department','region','education','gender','recruitment_channel']).fit(X_train,y_train)


data = ord_enc.transform(data)
data.head()

main_cols =data.columns.difference(['is_promoted'])



In [ ]:


train = data[:ntrain].copy()
#train.drop_duplicates(inplace = True, ignore_index=True)
target = train.is_promoted.copy()
train.drop('is_promoted', axis=1, inplace=True)

test = data[ntrain:].copy()
test.drop('is_promoted', axis=1, inplace=True)
test = test.reset_index(drop=True)

In [ ]:
class func() :   
    def __init__(self, train, label, test, model, model_type, random_state):
        self.train, self.label, self.test = train, label, test
        self.model, self.model_type = model, model_type
        self.random_state = random_state
        
        assert self.model_type in ('catboost', 'xgboost', 'lgbm'), 'Incorrect model_type'
    def __call__(self, plot = True):
        return self.fit(plot)

    def fit(self, plot):
        def catboost_fit(X_train, X_test, y_train, y_test):
            self.model.fit(X_train,y_train,eval_set=[(X_test,y_test)],early_stopping_rounds=500,
                           verbose=50,use_best_model=True)
            x_test_predict = self.model.predict_proba(X_test)[:,1]
            x_train_predict = self.model.predict_proba(X_train)[:,1]
            self.val_p[test_index] = x_test_predict
            self.test_p += self.model.predict_proba(self.test)[:,1]
            return x_test_predict, x_train_predict

        def xgboost_fit(X_train, X_test, y_train, y_test):
            self.model.fit(X_train, y_train, early_stopping_rounds = 30, eval_metric="auc",
                           eval_set=[(X_test, y_test)], verbose = True)
            x_test_predict = self.model.predict_proba(X_test, ntree_limit = self.model.get_booster().best_ntree_limit)[:,1]
            x_train_predict = self.model.predict_proba(X_train, ntree_limit = self.model.get_booster().best_ntree_limit)[:,1]
            self.val_p[test_index] = x_test_predict
            self.test_p += self.model.predict_proba(self.test, ntree_limit = self.model.get_booster().best_ntree_limit)[:,1]
            return x_test_predict, x_train_predict

        def lgbm_fit(X_train, X_test, y_train, y_test):
            self.model.fit(X_train, y_train, early_stopping_rounds = 30, eval_metric="auc",
                           eval_set=[(X_test, y_test)], verbose = True)
            x_test_predict = self.model.predict_proba(X_test, num_iteration = self.model.best_iteration_)[:,1]
            x_train_predict = self.model.predict_proba(X_train, num_iteration = self.model.best_iteration_)[:,1]
            self.val_p[test_index] = x_test_predict
            self.test_p += self.model.predict_proba(self.test, num_iteration = self.model.best_iteration_)[:,1]
            return x_test_predict, x_train_predict


        self.val_p = np.zeros(self.train.shape[0])
        mean_val = []
        mean_train = []
        self.test_p = np.zeros(self.test.shape[0])
        splits = 5
        kf = StratifiedKFold(n_splits = splits)
        for fold_count, (train_index, test_index) in enumerate(kf.split(self.train, self.label)):
            X_train,X_test = self.train.iloc[train_index],self.train.iloc[test_index]
            y_train,y_test = self.label.iloc[train_index],self.label.iloc[test_index]

            print(f"================================Fold{fold_count+1}====================================")
            if self.model_type == 'catboost': x_test_predict, x_train_predict = catboost_fit(X_train, X_test, y_train, y_test)
            elif self.model_type == 'xgboost': x_test_predict, x_train_predict = xgboost_fit(X_train, X_test, y_train, y_test)
            elif self.model_type == 'lgbm': x_test_predict, x_train_predict = lgbm_fit(X_train, X_test, y_train, y_test)

            print('\nValidation scores', roc_auc_score(y_test, x_test_predict), log_loss(y_test, x_test_predict))
            print('Training scores', roc_auc_score(y_train, x_train_predict), log_loss(y_train, x_train_predict))
            mean_val.append(roc_auc_score(y_test, x_test_predict))
            mean_train.append(roc_auc_score(y_train, x_train_predict))

        if plot:
            feat_imp = pd.DataFrame(sorted(zip(self.model.feature_importances_,self.train.columns)), columns=['Value','Feature'])
            plt.figure(figsize=(30,25))
            sns.barplot(x="Value", y="Feature", data=feat_imp.sort_values(by="Value", ascending=False))
            plt.ylabel('Feature Importance Score')
            plt.show()
        print(np.mean(mean_val), np.mean(mean_train), np.std(mean_val))
        return self.val_p, self.test_p/splits, self.model

In [ ]:
train.head()

In [ ]:
lgb_model = LGBMClassifier(objective =  'binary', 
                            metric= 'auc',
                            boosting_type= 'gbdt')

func_= func(train, target, test, lgb_model, 'lgbm', 1000)
val_p3, test_p3, model3 = func_()

In [ ]:

catboost = CatBoostClassifier(n_estimators=10000, eval_metric='AUC')

func_= func(train, target, test, catboost, 'catboost', 1000)
val_p1, test_p1, model1 = func_()

In [ ]:

xgboost = XGBClassifier()

func_= func(train, target, test, xgboost, 'xgboost', 1000)
val_p2, test_p2, model2 = func_()

In [ ]:
from sklearn.linear_model import  LinearRegression, Ridge, Lasso
stack = np.column_stack((val_p1, val_p2, val_p3))
stack_p = np.column_stack((test_p1, test_p2, test_p3))
predict = LinearRegression().fit(stack, target).predict(stack_p)

In [ ]:
submission['is_promoted'] = predict

submission.to_csv('final_predictions.csv',index=False)